# Step 1: Import libraries and directories

In [22]:
# Import required libraries
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path

# Step 2: Import the OHLC dataset into Panadas Dataframe

In [23]:
# Import the OHLCV dataset into a Pandas Dataframe
tick_df = pd.read_csv(
    Path("../Resources/META.csv"), 
    index_col='Date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
tick_df.head()

C:\Users\b_key\AppData\Local\Temp\ipykernel_15748\4067310274.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  tick_df = pd.read_csv(


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-19,141.210007,144.910004,132.500000,133.240005,133.240005,57404900
2018-12-20,130.699997,135.570007,130.000000,133.399994,133.399994,40297900
2018-12-21,133.389999,134.899994,123.419998,124.949997,124.949997,56901500
2018-12-24,123.099998,129.740005,123.019997,124.059998,124.059998,22066000
2018-12-26,126.000000,134.240005,125.889999,134.179993,134.179993,39723400


In [24]:
# Filter the date index and close columns
profit_df = tick_df.loc[:, ["High", "Low", "Close"]]

# Use the pct_change function to generate returns from close prices
profit_df["Actual Returns"] = profit_df["Close"].pct_change()

# Drop all NaN values from the DataFrame
profit_df = profit_df.dropna()

# Review the DataFrame
display(profit_df.head())
display(profit_df.tail())

,High,Low,Close,Actual Returns
Date,,,,
2018-12-20,135.570007,130.000000,133.399994,0.001201
2018-12-21,134.899994,123.419998,124.949997,-0.063343
2018-12-24,129.740005,123.019997,124.059998,-0.007123
2018-12-26,134.240005,125.889999,134.179993,0.081573
2018-12-27,134.990005,129.669998,134.520004,0.002534


,High,Low,Close,Actual Returns
Date,,,,
2023-12-12,334.470001,324.559998,334.220001,0.027484
2023-12-13,338.369995,332.640015,334.739990,0.001556
2023-12-14,334.700012,328.640015,333.170013,-0.004690
2023-12-15,338.660004,331.220001,334.920013,0.005253
2023-12-18,347.559998,337.019989,344.619995,0.028962


# Step 3: Generate the Stochastich Oscillator Technical Indicators

In [25]:
# def generate_stochastic_signals(df, high_col='High', low_col='Low', close_col='Close', k_period=14, d_period=3, overbought=80, oversold=20):
    # tick_df = generate_stochastic_signals(tick_df)
k_period=14
d_period=3
overbought=80
oversold=20

# Calculate %K and %D
profit_df['%K'] = 100 * ((profit_df['Close'] - profit_df['Low'].rolling(window=k_period).min()) / (profit_df['High'].rolling(window=k_period).max() - profit_df['Low'].rolling(window=k_period).min()))
profit_df['%D'] = profit_df['%K'].rolling(window=d_period).mean()

# Generate entry/exit signals
profit_df['Long Signal'] = (profit_df['%K'] < oversold) & (profit_df['%D'] < oversold)
profit_df['Short Signal'] = (profit_df['%K'] > overbought) & (profit_df['%D'] > overbought)

#Review dataframe
display(profit_df.head())
display(profit_df.tail())

,High,Low,Close,Actual Returns,%K,%D,Long Signal,Short Signal
Date,,,,,,,,
2018-12-20,135.570007,130.000000,133.399994,0.001201,NaN,NaN,False,False
2018-12-21,134.899994,123.419998,124.949997,-0.063343,NaN,NaN,False,False
2018-12-24,129.740005,123.019997,124.059998,-0.007123,NaN,NaN,False,False
2018-12-26,134.240005,125.889999,134.179993,0.081573,NaN,NaN,False,False
2018-12-27,134.990005,129.669998,134.520004,0.002534,NaN,NaN,False,False


,High,Low,Close,Actual Returns,%K,%D,Long Signal,Short Signal
Date,,,,,,,,
2023-12-12,334.470001,324.559998,334.220001,0.027484,70.266544,58.407350,False,False
2023-12-13,338.369995,332.640015,334.739990,0.001556,74.751774,61.577069,False,False
2023-12-14,334.700012,328.640015,333.170013,-0.004690,74.352197,73.123505,False,False
2023-12-15,338.660004,331.220001,334.920013,0.005253,81.021407,76.708459,False,False
2023-12-18,347.559998,337.019989,344.619995,0.028962,91.327423,82.233676,False,True


In [26]:
profit_df["Signals"] = np.where(profit_df['Short Signal'], -1,       
np.where(profit_df['Long Signal'], 1,0))

profit_df

,High,Low,Close,Actual Returns,%K,%D,Long Signal,Short Signal,Signals
Date,,,,,,,,,
2018-12-20,135.570007,130.000000,133.399994,0.001201,NaN,NaN,False,False,0
2018-12-21,134.899994,123.419998,124.949997,-0.063343,NaN,NaN,False,False,0
2018-12-24,129.740005,123.019997,124.059998,-0.007123,NaN,NaN,False,False,0
2018-12-26,134.240005,125.889999,134.179993,0.081573,NaN,NaN,False,False,0
2018-12-27,134.990005,129.669998,134.520004,0.002534,NaN,NaN,False,False,0
...,...,...,...,...,...,...,...,...,...
2023-12-12,334.470001,324.559998,334.220001,0.027484,70.266544,58.407350,False,False,0
2023-12-13,338.369995,332.640015,334.739990,0.001556,74.751774,61.577069,False,False,0
2023-12-14,334.700012,328.640015,333.170013,-0.004690,74.352197,73.123505,False,False,0


In [27]:
profit_df['Signals'].value_counts()

Signals
 0    863
-1    264
 1    130
Name: count, dtype: int64

In [28]:
# Calculate the strategy returns and add them to the signals_df DataFrame
profit_df['Strategy Returns'] = profit_df['Actual Returns'] * profit_df['Signals'].shift()

# Review the DataFrame
display(profit_df.head())
display(profit_df.tail())

,High,Low,Close,Actual Returns,%K,%D,Long Signal,Short Signal,Signals,Strategy Returns
Date,,,,,,,,,,
2018-12-20,135.570007,130.000000,133.399994,0.001201,NaN,NaN,False,False,0,NaN
2018-12-21,134.899994,123.419998,124.949997,-0.063343,NaN,NaN,False,False,0,-0.0
2018-12-24,129.740005,123.019997,124.059998,-0.007123,NaN,NaN,False,False,0,-0.0
2018-12-26,134.240005,125.889999,134.179993,0.081573,NaN,NaN,False,False,0,0.0
2018-12-27,134.990005,129.669998,134.520004,0.002534,NaN,NaN,False,False,0,0.0


,High,Low,Close,Actual Returns,%K,%D,Long Signal,Short Signal,Signals,Strategy Returns
Date,,,,,,,,,,
2023-12-12,334.470001,324.559998,334.220001,0.027484,70.266544,58.407350,False,False,0,0.0
2023-12-13,338.369995,332.640015,334.739990,0.001556,74.751774,61.577069,False,False,0,0.0
2023-12-14,334.700012,328.640015,333.170013,-0.004690,74.352197,73.123505,False,False,0,-0.0
2023-12-15,338.660004,331.220001,334.920013,0.005253,81.021407,76.708459,False,False,0,0.0
2023-12-18,347.559998,337.019989,344.619995,0.028962,91.327423,82.233676,False,True,-1,0.0


In [29]:
# Plot Strategy Returns to examine performance
(1 + profit_df[['Actual Returns', 'Strategy Returns']]).cumprod().hvplot(title='Actual Returns vs. Strategy Returns', ylabel='Returns')

:NdOverlay   [Variable]
   :Curve   [Date]   (value)